In [ ]:
import ipynb.fs.defs.preprocessing as preprocessing

import numpy as np

from pandas import DataFrame, concat

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
import xgboost as xgb

from matplotlib import pyplot

import warnings

warnings.filterwarnings('ignore')


In [2]:
data = preprocessing.DataProcessing()
data.build_datasets()

sliced eth data shape (312, 16)
sliced ae data shape (312, 8)
sliced eth data shape (411, 16)
sliced bat data shape (411, 8)
sliced eth data shape (369, 16)
sliced fun data shape (369, 8)
sliced eth data shape (441, 16)
sliced gno data shape (441, 8)
sliced eth data shape (595, 16)
sliced gnt data shape (595, 8)
sliced eth data shape (119, 16)
sliced loom data shape (119, 8)
sliced eth data shape (367, 16)
sliced omg data shape (367, 8)
sliced eth data shape (346, 16)
sliced rep data shape (346, 8)
sliced eth data shape (292, 16)
sliced salt data shape (292, 8)
sliced eth data shape (384, 16)
sliced snt data shape (384, 8)
sliced eth data shape (336, 16)
sliced zrx data shape (336, 8)
ae NaN Cells: 0
bat NaN Cells: 0
fun NaN Cells: 0
gno NaN Cells: 0
gnt NaN Cells: 0
loom NaN Cells: 0
omg NaN Cells: 0
rep NaN Cells: 0
salt NaN Cells: 0
snt NaN Cells: 0
zrx NaN Cells: 0
ae outperformed ETH on 145 / 312 days
46.47435897435898 % of the time
bat outperformed ETH on 190 / 411 days
46.228710

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.normalized_data['rep']['features'], data.normalized_data['rep']['targets'], test_size=0.33)


In [ ]:
model = XGBClassifier()
model.fit(X_train, y_train)

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# from sklearn.grid_search import GridSearchCV

xgb_model = xgb.XGBClassifier()
optimization_dict = {'max_depth': [4,5,6],
                     'n_estimators': [400,500,600,700],
                     'booster' : ['gblinear', 'dart']}

model = GridSearchCV(xgb_model, optimization_dict, 
                     scoring='accuracy', verbose=1)


model.fit(X_train, y_train)

print(model.best_score_)
print(model.best_params_)


In [ ]:
model.best_estimator_

In [ ]:
model.predict(X_test)

In [ ]:
model_comparrison = DataFrame(columns= ['xgboost', 'lstm', 'svm', 'decison_tree', 'MLP', ''])

In [ ]:
def process_model(model, X_train, X_test, y_train, y_test):
    
    eval_set = [(X_test, y_test)]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    X_pred = model.predict(X_train)
    test_accuracy = accuracy_score(y_test, predictions)
    train_accuracy = accuracy_score(y_train, X_pred)
    print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
    print("Train Accuracy: %.2f%%" % (train_accuracy * 100.0))
    y_proba = model.predict_proba(X_test) 
    return accuracy, y_pred, y_proba 

In [ ]:
process_model(xgb_model, X_train, X_test, y_train, y_test)

In [ ]:
eval_set = [(X_train, y_train), (X_test, y_test)]

# model= XGBClassifier()
model.best_estimator_.fit(X_train, y_train, eval_metric=["error", "logloss"], eval_set=eval_set, verbose=True)
    
y_pred = model.best_estimator_.predict(X_test)
predictions = [round(value) for value in y_pred]
X_pred = model.best_estimator_.predict(X_train)
test_accuracy = accuracy_score(y_test, predictions)
train_accuracy = accuracy_score(y_train, X_pred)
print("Test Accuracy: %.2f%%" % (test_accuracy * 100.0))
print("Train Accuracy: %.2f%%" % (train_accuracy * 100.0))
y_proba = model.best_estimator_.predict_proba(X_test) 
results = model.best_estimator_.evals_result()
# print(results)
epochs = len(results['validation_0']['error'])
x_axis = range(0, epochs)
# plot log loss
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
# plot classification error
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['error'], label='Train')
ax.plot(x_axis, results['validation_1']['error'], label='Test')
ax.legend()
pyplot.ylabel('Classification Error')
pyplot.title('XGBoost Classification Error')
pyplot.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D
from keras.layers import Dropout
from keras.layers import LSTM

X_train, X_test, y_train, y_test = data.train_test_sets['gnt']['train_X'], data.train_test_sets['gnt']['test_X'], data.train_test_sets['gnt']['train_y'], data.train_test_sets['gnt']['test_y']
                                            


model = Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Flatten())
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32))
model.add(Dropout(0.4))
model.add(Dense(32))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



# fit network
history = model.fit(X_train, y_train, epochs=2000, batch_size=10, validation_data=(X_test, y_test), verbose=1, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

pyplot.plot(history.history['acc'], label='train')
pyplot.plot(history.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()

Train on 250 samples, validate on 342 samples
Epoch 1/2000
250/250 [==============================] - 6s 26ms/step - loss: 0.6915 - acc: 0.5480 - val_loss: 0.6883 - val_acc: 0.5789
Epoch 2/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6894 - acc: 0.5480 - val_loss: 0.6850 - val_acc: 0.5789
Epoch 3/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6884 - acc: 0.5480 - val_loss: 0.6829 - val_acc: 0.5789
Epoch 4/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6883 - acc: 0.5480 - val_loss: 0.6817 - val_acc: 0.5789
Epoch 5/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6872 - acc: 0.5480 - val_loss: 0.6811 - val_acc: 0.5789
Epoch 6/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6896 - acc: 0.5480 - val_loss: 0.6809 - val_acc: 0.5789
Epoch 7/2000
250/250 [==============================] - 0s 1ms/step - loss: 0.6874 - acc: 0.5480 - val_loss: 0.6805 - val_acc: 0.5789
Epoch 8/2000
25